# 1. Проблема

![task4](task4.jpeg)

# 2. Содержательная постановка

1) Необходимо рассчитать оптимальный объем производства, при котором достигается максимальная прибыль, с использованием запаса на складе.

2) На основе решения прошлого пункта, выявить в каком случае прибыль максимальна:
  - Не покупать дополнительные ресурсы
  - Купить две единицы $i$ ресурса

# 3. Формальная постановка

Пусть 

- $X = \{x_1,x_2,x_3, x_4\}$ - пространство количества продуктов, произведенных по технологии $i$.

- $C = \{c_1,c_2,c_3, c_4\}$ - пространство весов (прибыли) производства по технологии $i$.

- $S = \{s_1,s_2,s_3\}$ - пространство запаса ресурсов на складе

- $A = ||a_{ij}||,\ \ i \in \{1,2,3\},\ j \in \{1,2,3,4\}$ - пространство затрат ресурсов на реализацию производства по технологии $j$

- $\textrm{CP} = \{\textrm{cp}_1,\textrm{cp}_2,\textrm{cp}_3\}$ - пространство себестоимостей ресурсов

Задача:

$
-(C,X) = - \sum\limits_{i=1}^4c_ix_i = - (c_1x_1 + c_2x_2 + c_3x_3 + c_4x_4) \rightarrow \min
$

Ограничения запаса ресурсов:

$
\bigg[\sum\limits_{i=1}^4 x_i \times a_{ij}\bigg]_j \leq s_j,\ \ j \in \{1,2,3\}
$

Естественные ограничения:

$
X \geq 0
$

# 4. Алгоритм и ПО

В качестве ПО будем использовать ЯП **Python** с подключенными модулями:

- **numpy** - для работы с линейной алгеброй

- **cvxpy** - для работы с линейным программированием

В качестве среды разработки **Jupyter Lab**,

# 5. Решение задачи

Приведем решение задачи с применением выбранного алгоритма.

Функция для решения задачи оптимизации:

In [1]:
def optimal_receipt(c,s,a):
    '''
    Функция, возвращающая оптимальное
    распределение производства
    по технологиям
    
    c - список 
    прибыли по технологиям
    
    s - список
    остатка на складе
    
    a - 2D-список
    ресурсных затрат

    '''
    
    import cvxpy
    import numpy as np
    from warnings import warn
    
    for i in c:
        if i < 0:
            warn('Одно или несколько значений прибыли отрицательное!')
    for i in a:
        for j in i:
            if j < 0:
                raise Exception('Затраты по ресурсам не могут быть отрицательными!')
    for i in s:
        if i < 0:
            raise Exception('Остаток на складе не может быть отрицательным!')
    
    c = np.array(c)
    a = np.array(a)
    s = np.array(s)

    x = cvxpy.Variable(shape=len(c), integer = True)
    
    constraints = [(x @ a.T <= s),
                   (x >= 0)]
               
    total_value = - c @ x
    problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=constraints)
    profit = abs(round(problem.solve()))
    remain = s - x.value @ a.T
    
    print('Для достижения максимальной прибыли в {}$, необходимо:\n'.format(profit))
    for i in range(len(x.value)):
        print('по технологии {} произвести {}\t продуктов'.format(i+1,round(x.value[i])))
    print('')
    for i in range(len(s)):
        print('Остаток {} ресурса: {}'.format(i+1, round(remain[i])))
    
    return profit

Функция для решения задачи о целесообразности покупки какого-либо вида ресурса в количестве $n$ шт:

In [2]:
def with_n_additional_resources(c,s,a,cp, n=2):
    '''
    Функция, решающая задачу о
    целесообразности покупки
    какого-либо вида ресурса в количестве
    n шт.
    
    c - список 
    прибыли по технологиям
    
    s - список
    остатка на складе
    
    a - 2D-список
    ресурсных затрат
    profits = [optimal_receipt(c,s,a)]
    
    cp - список
    себестоимостей ресурсов
    
    n - число
    покупаемых ресурсов
    '''
    
    print('В случае без покупки ресурсов:\n')
    profits = [optimal_receipt(c,s,a)]
    
    for i in range(len(s)):
        print('________________\n\nВ случае приобретения {} единиц {} ресурса:\n'.format(n, i+1))
        s_new = s
        s_new[i] += n
        profits.append(optimal_receipt(c,s_new,a) - n*cp[i])
    
    max_index = max(enumerate(profits),key=lambda x: x[1])[0]
    
    print('______________________________\nИтого:')
    if max_index == 0:
        print('Смысла в покупке дополнительных ресурсов нет')
    else:
        print('Лучше всего купить {} единиц {} ресурса.'.format(n, max_index))
        print('Это даст дополнительную прибыль в виде {}$.'.format(round(profits[max_index]-profits[0],1)))
    
    return(profits)

# 6. Анализ

**1)** Запустим функцию на наших данных:

In [3]:
s = [300,400,450]
c = [15,20,17,21]
a = [[0,3,9,11],
     [3,5,7,0],
     [4,8,0,13]]

optimal_receipt(c,s,a);

Для достижения максимальной прибыли в 1833$, необходимо:

по технологии 1 произвести 112	 продуктов
по технологии 2 произвести 0	 продуктов
по технологии 3 произвести 9	 продуктов
по технологии 4 произвести 0	 продуктов

Остаток 1 ресурса: 219
Остаток 2 ресурса: 1
Остаток 3 ресурса: 2


Получаем, что для наших данных, целесообразно использовать только две технологии: **первую** и **третью**.

Видим, что у нас остается много лишних ресурсов $1$, а также, что **третья** технология используется очень мало.

Следовательно, следует снизить закуп $1$ ресурса, отказаться от **второй** и **четвертой** технологий производства.

**2)** Проведем анализ целесообразности покупки дополнительных двух единиц какого-либо ресурса.

Запустим программу на наших данных:

In [4]:
s = [300,400,450]
c = [15,20,17,21]
a = [[0,3,9,11],
     [3,5,7,0],
     [4,8,0,13]]
cp = [0.18,0.22,0.19]
n = 2

with_n_additional_resources(c,s,a,cp, n);

В случае без покупки ресурсов:

Для достижения максимальной прибыли в 1833$, необходимо:

по технологии 1 произвести 112	 продуктов
по технологии 2 произвести 0	 продуктов
по технологии 3 произвести 9	 продуктов
по технологии 4 произвести 0	 продуктов

Остаток 1 ресурса: 219
Остаток 2 ресурса: 1
Остаток 3 ресурса: 2
________________

В случае приобретения 2 единиц 1 ресурса:

Для достижения максимальной прибыли в 1833$, необходимо:

по технологии 1 произвести 112	 продуктов
по технологии 2 произвести 0	 продуктов
по технологии 3 произвести 9	 продуктов
по технологии 4 произвести 0	 продуктов

Остаток 1 ресурса: 221
Остаток 2 ресурса: 1
Остаток 3 ресурса: 2
________________

В случае приобретения 2 единиц 2 ресурса:

Для достижения максимальной прибыли в 1836$, необходимо:

по технологии 1 произвести 106	 продуктов
по технологии 2 произвести 0	 продуктов
по технологии 3 произвести 12	 продуктов
по технологии 4 произвести 2	 продуктов

Остаток 1 ресурса: 172
Остаток 2 ресурса: 0
Остаток 

Видим, что первого ресурса докупать две единицы смысла нет. Прибыль та же.

Есть смысл купить две единицы третьего ресурса ресурса. Это повысит прибыль на $14.6$\$.